# Metadata Refresh for Snowflake


## Insert Server Name (if needed)

> The 'Metadata Server Name' is a **Friendly** name. It does not need to be the same as the _true_ Server name (which is determined by the 'Connection' selected at runtime)

In [1]:
DECLARE @server_name NVARCHAR(128) = /*${snowflake_Metadata_Server_Name}*/'snowflake_dev'/**/

IF NOT EXISTS (SELECT 1 FROM [meta].[dim_server] WHERE [server_name] = @server_name)
	BEGIN
		INSERT INTO [meta].[dim_server]
				   ([server_name]
				   ,[server_type]
				   )
			 VALUES
				   ( @server_name
				   , 'snowflake')
	END;

SELECT @@ROWCOUNT AS [row_count];


(1 row affected)

Total execution time: 00:00:00.060

row_count
0


## Truncate staging tables

> Reusing 'Truncate staging tables' step from [Metadata Refresh for SQL Server (Project)](.\metadata_refresh_for_sql_server_project.ipynb)

## Retrieve all Databases 
> Test the [refresh_database_names_snowflake](.\content\refresh_database_names_snowflake.sql)

## Note about building project to run dataflow before testing merge statement

## Merge stg to meta schema - dim\_database (snowflake)

In [ ]:
DELETE [meta].[dim_database]
  FROM [meta].[dim_database] AS e
  LEFT JOIN [stg].[dim_database] AS s
    ON s.[server_name] = e.[server_name]
   AND s.[database_name] = e.[database_name]
 WHERE e.[server_name] = '${snowflake_Metadata_Server_Name}'
   AND s.[database_name] IS NULL

INSERT [meta].[dim_database]
SELECT s.[server_name]
	 , s.[database_name]
	 , s.[database_create_date]
	 , s.[change_tracking_enabled]
     , 'N'
     , 'N'
  FROM [stg].[dim_database] AS s
  LEFT JOIN [meta].[dim_database] AS e
    ON e.[server_name] = s.[server_name]
   AND e.[database_name] = s.[database_name]
 WHERE s.[server_name] = '${snowflake_Metadata_Server_Name}'
   AND e.[database_name] IS NULL;

SELECT @@ROWCOUNT AS [row_count];

## src to stg - SQL Server tables

In [ ]:
SELECT '${snowflake_Metadata_Server_Name}'::VARCHAR(128) AS server_name
, TABLE_CATALOG::VARCHAR(128) AS database_name
, TABLE_SCHEMA::VARCHAR(128) AS table_schema
, TABLE_NAME::VARCHAR(128) AS table_name
, TABLE_TYPE::VARCHAR(32) AS table_type
, 'N'::VARCHAR(1) AS has_identity
, 'N'::VARCHAR(1) AS has_primary_key
, 'SQL'::VARCHAR(16) AS row_data_source
 FROM ${DEV_RTI_CURATED_Database}.INFORMATION_SCHEMA.TABLES 
 WHERE TABLE_SCHEMA != 'INFORMATION_SCHEMA';

## src to stg - SQL Server columns

In [ ]:
DECLARE @Metadata_Server_Name SYSNAME = /*${Metadata_Server_Name}*/'dev'/**/;

WITH [pk_columns] AS
(
SELECT tc.[TABLE_CATALOG] AS [database_name]
	 , tc.[TABLE_SCHEMA] AS [table_schema]
	 , tc.[TABLE_NAME] AS [table_name]
	 , cc.[COLUMN_NAME] AS [column_name]
  FROM INFORMATION_SCHEMA.TABLE_CONSTRAINTS tc
  JOIN INFORMATION_SCHEMA.CONSTRAINT_COLUMN_USAGE cc 
		ON tc.CONSTRAINT_NAME = cc.Constraint_name
    WHERE tc.CONSTRAINT_TYPE = 'Primary Key'
)
SELECT DISTINCT @Metadata_Server_Name AS [server_name]
	 , c.[TABLE_CATALOG] AS [database_name] 
	 , c.[TABLE_SCHEMA] AS [table_schema] 
	 , c.[TABLE_NAME] AS [table_name] 
	 , c.[COLUMN_NAME] AS [column_name] 
	 , c.[ORDINAL_POSITION] AS [ordinal_position] 
	 , CAST(c.[IS_NULLABLE] AS NVARCHAR(3)) AS [is_nullable] 
	 , c.[DATA_TYPE] AS [data_type] 
	 , c.[CHARACTER_MAXIMUM_LENGTH] AS [char_max_length] 
	 , CASE WHEN p.[table_name] IS NULL
		    THEN N'N'
			ELSE N'Y'
		END AS [primary_key]
	 , N'SQL' AS [row_data_source]
FROM INFORMATION_SCHEMA.COLUMNS c 
  LEFT JOIN [pk_columns] p
    ON p.[database_name] = c.[TABLE_CATALOG]
   AND p.[table_schema] = c.[TABLE_SCHEMA]
   AND p.[table_name] = c.[TABLE_NAME]
   AND p.[column_name] = c.[COLUMN_NAME];

## Merge - stg to meta - dim_table

In [ ]:
MERGE [meta].[dim_table] AS DST
USING [stg].[dim_table] AS SRC
   ON src.[server_name] = dst.[server_name]
   AND src.[database_name] = dst.[database_name]
   AND src.[table_schema] = dst.[table_schema]
   AND src.[table_name] = dst.[table_name]

WHEN MATCHED AND (    DST.[table_type] != SRC.[table_type] 
                   OR DST.[has_identity] != SRC.[has_identity] 
	           OR DST.[has_primary_key] != SRC.[has_primary_key] ) THEN
		UPDATE SET [table_type] =  SRC.[table_type] 
                 , [has_identity] =  SRC.[has_identity] 
                 , [has_primary_key] = SRC.[has_primary_key] 

WHEN NOT MATCHED THEN 
INSERT ( [server_name]
      ,[database_name]
      ,[table_schema]
      ,[table_name]
      ,[table_type]
      ,[has_identity]
      ,[has_primary_key]
      ,[enable_change_tracking]
	   )
VALUES (
	   [server_name]
      ,[database_name]
      ,[table_schema]
      ,[table_name]
      ,[table_type]
      ,[has_identity]
      ,[has_primary_key]
      ,'N'	
	   )
WHEN NOT MATCHED BY SOURCE 
 AND DST.[server_name] = /*${Metadata_Server_Name}*/'dev'/**/ 
 AND DST.[database_name] = /*${sql_server_Database}*/'eltsnap_v2'/**/ THEN
	 DELETE;

SELECT @@ROWCOUNT AS [row_count];

## Merge - stg to meta - dim_column

In [ ]:
MERGE [meta].[dim_column] AS DST
USING [stg].[dim_column] AS SRC
   ON src.[server_name] = dst.[server_name]
   AND src.[database_name] = dst.[database_name]
   AND src.[table_schema] = dst.[table_schema]
   AND src.[table_name] = dst.[table_name]
   AND src.[column_name] = dst.[column_name]

WHEN MATCHED AND (    DST.[ordinal_position] != SRC.[ordinal_position] 
                   OR DST.[is_nullable] != SRC.[is_nullable] 
		   OR DST.[data_type] != SRC.[data_type]
                   OR DST.[char_max_length] != SRC.[char_max_length] 
		   OR DST.[primary_key] != SRC.[primary_key] 
               ) THEN
UPDATE SET [ordinal_position] = SRC.[ordinal_position] 
         , [is_nullable] = SRC.[is_nullable] 
         , [data_type] = SRC.[data_type] 
         , [char_max_length] = SRC.[char_max_length]
		   , [primary_key] = SRC.[primary_key]

WHEN NOT MATCHED THEN 
INSERT (  [server_name]
	    , [database_name]
        , [table_schema]
        , [table_name]
        , [column_name]
        , [ordinal_position]
        , [is_nullable]
        , [data_type]
        , [char_max_length]
		, [primary_key]
	   )
VALUES (
		  [server_name]
	    , [database_name]
        , [table_schema]
        , [table_name]
        , [column_name]
        , [ordinal_position]
        , [is_nullable]
        , [data_type]
        , [char_max_length]
		, [primary_key]
	   )
WHEN NOT MATCHED BY SOURCE
 AND DST.[server_name] = /*${Metadata_Server_Name}*/'dev'/**/ 
 AND DST.[database_name] = /*${sql_server_Database}*/'eltsnap_v2'/**/ THEN
	 DELETE;

SELECT @@ROWCOUNT AS [row_count];

## Merge stg to dbo schema - dim_table

In [ ]:
EXEC [elt].[build_and_execute_merge_for_dimension] 'stg', 'dim_table', 'dbo', 'dim_table', 'Standard', 'server_name,database_name,table_schema,table_name','table_type';

## Merge stg to dbo schema - dim_column

In [ ]:
EXEC [elt].[build_and_execute_merge_for_dimension] 'stg', 'dim_column', 'dbo', 'dim_column', 'Standard', 'server_name,database_name,table_schema,table_name,column_name','is_nullable,data_type,char_max_length';

## dim delete - dbo - dim_table

In [ ]:
WITH [deleted_rows] AS
(
SELECT [server_name]
	 , [database_name]
	 , [table_schema]
	 , [table_name]
  FROM [dbo].[dim_table]
EXCEPT 
SELECT [server_name]
	 , [database_name]
	 , [table_schema]
	 , [table_name]
  FROM [meta].[dim_table]
)
UPDATE [dbo].[dim_table]
   SET [row_expiration_date] = CAST(GETDATE() AS DATE)
     , [row_update_date] = CAST(GETDATE() AS DATE)
  FROM [dbo].[dim_table] t
  JOIN [deleted_rows] d
    ON d.[server_name]   = t.[server_name]
   AND d.[database_name] = t.[database_name]
   AND d.[table_schema]  = t.[table_schema]
   AND d.[table_name]    = t.[table_name]
 WHERE t.[row_expiration_date] = '9999-12-31';

SELECT @@ROWCOUNT AS [row_count];

## dim delete - dbo - dim_column

In [ ]:
WITH [deleted_rows] AS
(
SELECT [server_name]
	 , [database_name]
	 , [table_schema]
	 , [table_name]
	 , [column_name]
  FROM [dbo].[dim_column]
EXCEPT 
SELECT [server_name]
	 , [database_name]
	 , [table_schema]
	 , [table_name]
	 , [column_name]
  FROM [meta].[dim_column]
)
UPDATE [dbo].[dim_column]
   SET [row_expiration_date] = CAST(GETDATE() AS DATE)
     , [row_update_date] = CAST(GETDATE() AS DATE)
  FROM [dbo].[dim_column] c
  JOIN [deleted_rows] d
    ON d.[server_name]   = c.[server_name]
   AND d.[database_name] = c.[database_name]
   AND d.[table_schema]  = c.[table_schema]
   AND d.[table_name]    = c.[table_name]
   AND d.[column_name]   = c.[column_name]
 WHERE c.[row_expiration_date] = '9999-12-31';

SELECT @@ROWCOUNT AS [row_count];